In [17]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv()
from langchain.chains import LLMChain
import json
from langchain.prompts import (
    ChatPromptTemplate,
)
from langchain.llms import OpenAI




In [18]:
# from langfuse.callback import CallbackHandler
# handler = CallbackHandler(os.environ.get("LF_PK"), os.environ.get("LF_SK"), os.environ.get("LF_HOST"))

In [19]:
GENERATIONS_PATH = "../datasets/baseline_zero_shot_gpt3_50_50_merged.csv"
MODEL1 = "gpt-3.5-turbo-0301"
MODEL2 = "gpt-3.5-turbo-instruct"

In [20]:
data = pd.read_csv(GENERATIONS_PATH)

if "whole_output" not in data.columns:
    data["whole_output"] = ""


if "similarity" not in data.columns:
    data["similarity"] = ""
if "confidence" not in data.columns:
    data["confidence"] = ""
data

,Unnamed: 0,traceId,model,startTime,endTime,prompt,completion,predicted_label,label,error,similarity,confidence,whole_output,structured_explanation
0,0,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:58.793Z,2023-10-22T13:05:59.340Z,Are the following two products the same?\nProd...,False.,False,True,FN,30%,50%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""..."
1,1,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:58.216Z,2023-10-22T13:05:58.791Z,Are the following two products the same?\nProd...,True,True,True,NONE,90%,80%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."
2,2,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:57.825Z,2023-10-22T13:05:58.212Z,Are the following two products the same?\nProd...,True,True,True,NONE,100%,90%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."
3,3,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:57.322Z,2023-10-22T13:05:57.823Z,Are the following two products the same?\nProd...,True,True,True,NONE,95%,90%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""..."
4,4,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:56.925Z,2023-10-22T13:05:57.320Z,Are the following two products the same?\nProd...,True,True,True,NONE,100%,90%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.2"",""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,216,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:03:55.654Z,2023-10-22T13:03:56.530Z,Are the following two products the same?\nProd...,False,False,False,NONE,0%,50%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""..."
217,217,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:03:55.074Z,2023-10-22T13:03:55.653Z,Are the following two products the same?\nProd...,False.,False,False,NONE,50%,70%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""..."
218,218,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:03:54.615Z,2023-10-22T13:03:55.071Z,Are the following two products the same?\nProd...,False.,False,False,NONE,10%,50%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."
219,219,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:03:54.077Z,2023-10-22T13:03:54.614Z,Are the following two products the same?\nProd...,False.,False,False,NONE,25%,50%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."


In [21]:


task = """ Provide a similarity score for the items and a confidence score for your decision as percentage values, 100% referring to perfect similarity or full confidence. 
Please provide your answer exactly in the following format which includes an example output:
Similarity: 75%
Confidence: 60%
"""


STEP_CONFIDENCE_PROMPT = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful AI."),
            ("human", "{user_prompt}"),
            ("ai", "{ai_answer}"),
            ("human", task),
        ])





In [22]:
missed = []
def get_confidence_score(start, end):
    if (MODEL1 == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name=MODEL1, temperature=0, max_tokens=100)
    else:
        llm = ChatOpenAI(model_name=MODEL1, temperature=0, max_tokens=100)
    chain = LLMChain(llm=llm, prompt=STEP_CONFIDENCE_PROMPT)
    for i in range(start, end):
        print(i)
        user_prompt = data["prompt"][i]
        ai_answer = data["completion"][i]
        try:
            output = str(chain.run(user_prompt=user_prompt, ai_answer=ai_answer))
        except:
            output = "error"
            i = i - 1
            continue
        print(STEP_CONFIDENCE_PROMPT.format(user_prompt=user_prompt, ai_answer=ai_answer))
        print("----")
        print(data["label"][i])
        print(output)
        print("-----------")
        splitted_output = output.split("\n")
        try:
            similarity = splitted_output[0].split(": ")[1]
            confidence = splitted_output[1].split(": ")[1]
            data["similarity"][i] = similarity
            data["confidence"][i] = confidence
            data["whole_output"][i] = output
        except:
            data["similarity"][i] = output
            data["confidence"][i] = output
            missed.append(i)



In [23]:
# get_confidence_score(0, len(data))

In [24]:
print(missed)

[]


In [25]:
data

,Unnamed: 0,traceId,model,startTime,endTime,prompt,completion,predicted_label,label,error,similarity,confidence,whole_output,structured_explanation
0,0,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:58.793Z,2023-10-22T13:05:59.340Z,Are the following two products the same?\nProd...,False.,False,True,FN,30%,50%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""..."
1,1,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:58.216Z,2023-10-22T13:05:58.791Z,Are the following two products the same?\nProd...,True,True,True,NONE,90%,80%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."
2,2,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:57.825Z,2023-10-22T13:05:58.212Z,Are the following two products the same?\nProd...,True,True,True,NONE,100%,90%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."
3,3,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:57.322Z,2023-10-22T13:05:57.823Z,Are the following two products the same?\nProd...,True,True,True,NONE,95%,90%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""..."
4,4,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:56.925Z,2023-10-22T13:05:57.320Z,Are the following two products the same?\nProd...,True,True,True,NONE,100%,90%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.2"",""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,216,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:03:55.654Z,2023-10-22T13:03:56.530Z,Are the following two products the same?\nProd...,False,False,False,NONE,0%,50%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""..."
217,217,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:03:55.074Z,2023-10-22T13:03:55.653Z,Are the following two products the same?\nProd...,False.,False,False,NONE,50%,70%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""..."
218,218,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:03:54.615Z,2023-10-22T13:03:55.071Z,Are the following two products the same?\nProd...,False.,False,False,NONE,10%,50%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."
219,219,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:03:54.077Z,2023-10-22T13:03:54.614Z,Are the following two products the same?\nProd...,False.,False,False,NONE,25%,50%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."


In [26]:
# data.to_csv(GENERATIONS_PATH, index=False)


In [27]:
task_structured_explanation = """Explain your decision in a structured format, listing the attributes that you compared for reaching your decision. Each attribute should be accompanied by the attribute values and a score between 0 and 1 that shows the importance of the attribute for the decision. If an attribute only occurs in one item, specify the value of that attribute for the other item as "missing". The attributes should not be just the Title or Description but more specific attributes like Brand, Model, Color, etc. 
The format should exactly match the following example output: 
[{"attribute":"brand","importance":"0.05","values":["Logitech","Logitech"]},{"attribute":"model","importance":"0.95" "values":["G500","MX Master 3S"]},{"attribute":"color","importance":"0.00" "values":["missing","Graphite"]}]"""

task_structured_explanation_v2 = """Explain your decision in a structured format, listing the attributes that you compared for reaching your decision. Each attribute should be accompanied by the attribute values and a score between 0 and 1 that shows the importance of the attribute for the decision. If an attribute only occurs in one item, specify the value of that attribute for the other item as "missing". The attributes should not be just the Title or Description but more specific attributes like Brand, Model, Color, etc. 
The format should exactly match the following example output: 
[{"attribute":"brand","importance":"0.05","values":["Logitech","Logitech"]},{"attribute":"model","importance":"0.95", "values":["G500","MX Master 3S"]},{"attribute":"color","importance":"0.00", "values":["missing","Graphite"]}]"""


task_structured_explanation_v3 = """Explain your decision in a structured format, listing the attributes that you compared for reaching your decision. Each attribute should be accompanied by the attribute values and a score between 0 and 1 that shows the importance of the attribute for the decision. If an attribute only occurs in one item, specify the value of that attribute for the other item as "missing". The attributes should not be just the Title or Description but more specific attributes like Brand, Model, Color, etc. 
The format should exactly match the following example output: 
attribute:brand, importance:0.05,values:[Logitech,Logitech]
attribute:model,importance:0.95,values:[G500,MX Master 3S]
attribute:color,importance:0.00,values:[missing,Graphite]"""




STEP_EXPLANATION_STRUCTURED_PROMPT = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful AI."),
            ("human", "{user_prompt}"),
            ("ai", "{ai_answer}"),
            ("human", "{task_structured_explanation}"),
        ])




In [28]:
if "structured_explanation" not in data.columns:
    data["structured_explanation"] = ""
if "structured_explanation_v2" not in data.columns:
    data["structured_explanation_v2"] = ""

In [29]:
def get_structured_explanation(start, end):
    if (MODEL2 == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name=MODEL2, temperature=0)
    else:
        llm = ChatOpenAI(model_name=MODEL2, temperature=0)
    chain = LLMChain(llm=llm, prompt=STEP_EXPLANATION_STRUCTURED_PROMPT,
    # callbacks=[handler]
    )
    for i in range(start, end):
        print(i)
        user_prompt = data["prompt"][i]
        ai_answer = data["completion"][i]
        try:
            output = chain.run(user_prompt=user_prompt, ai_answer=ai_answer, task_structured_explanation=task_structured_explanation_v2, 
            # callbacks=[handler]
            )
        except:
            output = "error"
            i = i - 1
            continue
        print(STEP_EXPLANATION_STRUCTURED_PROMPT.format(user_prompt=user_prompt, ai_answer=ai_answer, task_structured_explanation=task_structured_explanation))
        print("----")
        print(data["label"][i])
        print(output)
        print("-----------")
        data["structured_explanation_v2"][i] = output



In [30]:
get_structured_explanation(0, len(data))

0
System: You are a helpful AI.
Human: Are the following two products the same?
Product 1: Title: Traveler's Lined Notebook Refill - Regular Size
Description: Think about the days you went to school. Based on people’s past experience as a student, we receive comments saying that this is the most easy type of notebook to use. The lines are printed with an interval of 6.5mm, and the paper is our originally m
Product 2: Title: Traveler's Company Traveler's Company Notebook Accessories
Description: none

Answer with 'True' or 'False'.
AI: False.
Human: Explain your decision in a structured format, listing the attributes that you compared for reaching your decision. Each attribute should be accompanied by the attribute values and a score between 0 and 1 that shows the importance of the attribute for the decision. If an attribute only occurs in one item, specify the value of that attribute for the other item as "missing". The attributes should not be just the Title or Description but more sp

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_89002/995111640.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["structured_explanation_v2"][i] = output


System: You are a helpful AI.
Human: Are the following two products the same?
Product 1: Title: MINION DHR II 29 X 2.30 3C TERRA TR DD FOLD
Description: Refined and redesigned; ready to shred any lineRamped and siped tread to roll fast and track straight under brakingSingle-ply, DoubleDown, and DH casings available3C MaxxTerra, 3C MaxxGrip, and Super Tacky available on select modelsWide trail (WT) ca
Product 2: Title: Maxxis Minion DHR II 29\" DD/3C/TR
Description: Minion DHR II 29 x 2.35 tubeless ready tyre with Double down casing and 3C Maxx Terra.For our full range of Bike Accessories visit Evans Cycles

Answer with 'True' or 'False'.
AI: True
Human: Explain your decision in a structured format, listing the attributes that you compared for reaching your decision. Each attribute should be accompanied by the attribute values and a score between 0 and 1 that shows the importance of the attribute for the decision. If an attribute only occurs in one item, specify the value of that attrib

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_89002/995111640.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["structured_explanation_v2"][i] = output


System: You are a helpful AI.
Human: Are the following two products the same?
Product 1: Title: Traveler's Company 014. CRAFT PAPER (64PG) MIDORI TRAVELER'S NOTEBOOK
Description: none
Product 2: Title: Refill #014: Kraft Paper Notebook
Description: Cardboard Paper notebook refill for the Traveler's Notebook.Various cuttings, stickers, photos, scribbles and more sophisticated drawings suit it well.Contains 64 pages and its dimensions (210 x 110 mm) make it suitable for the classic version, not f

Answer with 'True' or 'False'.
AI: True
Human: Explain your decision in a structured format, listing the attributes that you compared for reaching your decision. Each attribute should be accompanied by the attribute values and a score between 0 and 1 that shows the importance of the attribute for the decision. If an attribute only occurs in one item, specify the value of that attribute for the other item as "missing". The attributes should not be just the Title or Description but more specific 

...

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_89002/995111640.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["structured_explanation_v2"][i] = output


System: You are a helpful AI.
Human: Are the following two products the same?
Product 1: Title: Instax Mini Rainbow Film 10 Pack
Description: Brighten the place up with this pack of rainbow coloured film! Top tip: one of these would look fab in our Instax Mini Acrylic Photo Block!Includes pack of 10 shots
Product 2: Title: 1 Fujifilm Instax Square Film Black Frame
Description: none

Answer with 'True' or 'False'.
AI: False.
Human: Explain your decision in a structured format, listing the attributes that you compared for reaching your decision. Each attribute should be accompanied by the attribute values and a score between 0 and 1 that shows the importance of the attribute for the decision. If an attribute only occurs in one item, specify the value of that attribute for the other item as "missing". The attributes should not be just the Title or Description but more specific attributes like Brand, Model, Color, etc. 
The format should exactly match the following example output: 
[{"attr

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_89002/995111640.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["structured_explanation_v2"][i] = output


System: You are a helpful AI.
Human: Are the following two products the same?
Product 1: Title: DJI Mavic Mini Drone Fly More Combo
Description: Mavic Mini, the everyday flycam. Open your sky with the compact, yet powerful Mavic Mini. Weighing under 250 grams, together with the DJI Fly APP, you'll enjoy a simplified flying experience that is almost as light as the average smartphone.
Product 2: Title: Fujifilm Instax Mini Instant Film - Monochrome (10 Shots)
Description: Monochrome (Black & White) Instant Film for Fujifilm Mini Instant Cameras10 Shots per PackISO 800High Gloss FinishSuperb Grain QualityFast Image FormationGreater Resistance to Age-Induced ChangesSingle-plastic Composition for Easier Disposal

Answer with 'True' or 'False'.
AI: False.
Human: Explain your decision in a structured format, listing the attributes that you compared for reaching your decision. Each attribute should be accompanied by the attribute values and a score between 0 and 1 that shows the importance of

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_89002/995111640.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["structured_explanation_v2"][i] = output


In [31]:
data

,Unnamed: 0,traceId,model,startTime,endTime,prompt,completion,predicted_label,label,error,similarity,confidence,whole_output,structured_explanation,structured_explanation_v2
0,0,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:58.793Z,2023-10-22T13:05:59.340Z,Are the following two products the same?\nProd...,False.,False,True,FN,30%,50%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""...","\n\n[{""attribute"":""brand"",""importance"":""0.5"",""..."
1,1,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:58.216Z,2023-10-22T13:05:58.791Z,Are the following two products the same?\nProd...,True,True,True,NONE,90%,80%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.05"",...","\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."
2,2,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:57.825Z,2023-10-22T13:05:58.212Z,Are the following two products the same?\nProd...,True,True,True,NONE,100%,90%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.05"",...","\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."
3,3,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:57.322Z,2023-10-22T13:05:57.823Z,Are the following two products the same?\nProd...,True,True,True,NONE,95%,90%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""...","\n\n[{""attribute"":""brand"",""importance"":""0.5"",""..."
4,4,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:56.925Z,2023-10-22T13:05:57.320Z,Are the following two products the same?\nProd...,True,True,True,NONE,100%,90%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.2"",""...","\n\n[{""attribute"":""brand"",""importance"":""0.1"",""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,216,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:03:55.654Z,2023-10-22T13:03:56.530Z,Are the following two products the same?\nProd...,False,False,False,NONE,0%,50%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""...","\n\n[{""attribute"":""brand"",""importance"":""0.5"",""..."
217,217,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:03:55.074Z,2023-10-22T13:03:55.653Z,Are the following two products the same?\nProd...,False.,False,False,NONE,50%,70%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""...","\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."
218,218,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:03:54.615Z,2023-10-22T13:03:55.071Z,Are the following two products the same?\nProd...,False.,False,False,NONE,10%,50%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.05"",...","\n\n[{""attribute"":""brand"",""importance"":""0.5"",""..."
219,219,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:03:54.077Z,2023-10-22T13:03:54.614Z,Are the following two products the same?\nProd...,False.,False,False,NONE,25%,50%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.05"",...","\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."


In [32]:
data.to_csv(GENERATIONS_PATH, index=False)
